In [162]:
import cloudscraper
from bs4 import BeautifulSoup
import math
import pprint
import time


# Takes a public Discogs store inventory and returns pricing information on other listings on the market.


scraper = cloudscraper.create_scraper(browser={'browser': 'firefox','platform': 'windows','mobile': False})


# Classes

# class listing:

class formattedListings:
    def __init__(self,title,url,listings,place,total):
        self.title = title
        self.url = url
        self.listings = listings
        self.place = place
        self.total = "Total: {0}".format(total)

    def __str__(self):
        return "{0}\n{1}\n{2}\n{3}".format(self.title,self.url,self.listings,self.total)

# Helper Functions

def get_inventory(username):

    URL = "https://www.discogs.com/seller/{0}/profile".format(username)
    pages = count_pages(URL)
    
    return parse_list(URL, pages)
    
def parse_list(URL, pages): # Takes URL of a store inventory, returns the releases as a list.
    
    new_list = []

    for page in range(1, pages + 1):
        new_URL = URL + "?&limit=250&sort=price&sort_order=asc&page={0}".format(page)
        html = scraper.get(new_URL).content
        soup = BeautifulSoup(html, 'html.parser')

        list_items = soup.find(id="pjax_container").find("tbody").find_all("tr")
        for item in list_items:
            release = item.find("td", class_="item_description")
            title = release.find("strong").text.strip()
            item_id = release.find("a", class_="item_release_link")["href"].split("-")[0].strip("/release/")

            new_list_item = (title, item_id)
            new_list.append(new_list_item)
    
    return new_list

def count_pages(URL): # Takes URL for a Discogs store, returns the number of pages.
    
    html = scraper.get(URL).content
    soup = BeautifulSoup(html, 'html.parser')
    
    inventory_size = int(soup.find(id="page_content").find("li", class_="first").find("h2").text.strip().strip("For Sale"))
    pages = math.ceil(inventory_size/250)
            
    return pages

def get_listings(inventory_list, username, release_title, item_id): # Given username and item_id, scrapes marketplace listings for prices.
    
    URL = "https://www.discogs.com/sell/release/{0}?ships_from=United+States&sort=price%2Casc".format(item_id)
    html = scraper.get(URL).content
    soup = BeautifulSoup(html, 'html.parser')
    
    count = 0
    your_place = 0
    formatted_listings = ""
    
    listings = soup.find("table", class_="mpitems").find_all("tr", class_="shortcut_navigable")
    for listing in listings:
        count += 1
        if is_user(username, listing):
            formatted_listings += "{0} (You) ({1})\n".format(get_price(listing), count)
            your_place = count
            # break
        else:
            if check_scam(listing):
                formatted_listings += "{0} (SCAM)\n".format(get_price(listing))
            else:
                formatted_listings += "{0}\n".format(get_price(listing))
                
    total = (soup.find("strong", class_="pagination_total").text.split(" of "))[-1]
    entry = formattedListings(release_title,URL,formatted_listings,your_place,total)
    
    if your_place < 10:
        (inventory_list[your_place - 1]).append(entry)
    else:
        inventory_list[9].append(entry)
                        
    return
    
def format_condition(item_condition): # Formats the item condition to (Media/Sleeve).
    
    # print(item_condition)
    
    media = (item_condition.split("("))[1].split(")")[0]
    try:
        sleeve = (item_condition.split("("))[2].split(")")[0]
    except IndexError:
        return "({0})".format(media.split(" or")[0])

    media = media.split(" or")[0]
    sleeve = sleeve.split(" or")[0]
    
    return "({0}/{1})".format(media, sleeve)

def get_price(listing): # Gets price provided a listing.

    try:
        item_condition = listing.find("p", class_="item_condition").text
        formatted_condition = format_condition(item_condition)
        
        if listing.find(string="New seller"):
            return "{0} {1} (New)".format(listing.find("span", class_="converted_price").text.strip(), formatted_condition)
        else:
            return "{0} {1}".format(listing.find("span", class_="converted_price").text.strip(), formatted_condition)
    except AttributeError:
        return "n/a"
    
def is_user(username, listing): # Checks if a marketplace listing matches the provided username.
    
    return listing.find(string=username)

def check_scam(listing): # Checks if a listing is a scam (has 0.0% seller rating).
    
    return listing.find(string="0.0%")


# Main

print("Enter username: ")
user1 = input()
print("\n")

try:
    start_time = time.time()
    inventory = get_inventory(user1)

    inventory_list = [[],[],[],[],[],[],[],[],[],[]]
    count = 0
    places = []
    
    for release in inventory:
        get_listings(inventory_list,user1,release[0],release[1])
         
    for place in inventory_list:
        counter = 0
        
        for entry in place:
            count += 1
            counter += 1
            print("({0})".format(count))
            print("{0}\n".format(entry))
            
        places.append(counter)
        print("Count: {0}".format(counter))
        print('─' * 50)
        
    print("Place")
    for place in range(len(places)):
        print("{0}: {1}".format(place+1, places[place]))
    
    print("\nDone.")
    end_time = time.time()
    print("Total time: {0} seconds".format(end_time-start_time))
        
except AttributeError:
    print("Can't find user.")


Enter username: 
curefortheitch


(1)
blackbear - Everything Means Nothing (LP, Album)
https://www.discogs.com/sell/release/15946789?ships_from=United+States&sort=price%2Casc
$48.99 total (NM/NM) (You) (1)
$50.50 total (VG+/NM)
$53.90 total (M/NM)
$59.50 total (M/M)

Total: 4    

(2)
Metric - Grow Up And Blow Away (LP, Album, RE)
https://www.discogs.com/sell/release/25845355?ships_from=United+States&sort=price%2Casc
$64.99 total (M/NM) (You) (1)
$65.50 total (M/M)

Total: 2    

(3)
Turnover (3) - Peripheral Vision (LP, Album, Ltd, RP, Red)
https://www.discogs.com/sell/release/9867657?ships_from=United+States&sort=price%2Casc
$64.99 total (VG+/VG+) (You) (1)
$79.99 total (NM/NM)
$85.50 total (NM/NM)

Total: 3    

(4)
Passion Pit - Manners (LP, Album, RE)
https://www.discogs.com/sell/release/10877330?ships_from=United+States&sort=price%2Casc
$68.99 total (M/NM) (You) (1)
$145.00 total (M/M) (New)

Total: 2    

(5)
Blackbear - Everything Means Nothing (LP, Album, Ltd, Bab)
https://www